In [ ]:
import numpy as np # linear algebra
import dicom
import os
import sys
from joblib import Parallel, delayed

In [ ]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def process_single_patient(patient_dir, INPUT_DICOM_DIR, OUTPUT_NPY_DIR):
    output_file_path = os.path.join(OUTPUT_NPY_DIR, patient_dir + '.npy')
    if not os.path.isfile(output_file_path):
        print ("Creating file", output_file_path)
        slices = load_scan(os.path.join(INPUT_DICOM_DIR, patient_dir))
        array = get_pixels_hu(slices)

        output_file_path = os.path.join(OUTPUT_NPY_DIR, patient_dir + '.npy')
        np.save(output_file_path, array)
    else:
        print(output_file_path + " already created.")

In [ ]:
INPUT_DICOM_DIR = '/notebooks/DSB3/data/stage2/'
OUTPUT_NPY_DIR = '/notebooks/DSB3/data/stage2_npy/'

Parallel(n_jobs = 10)(delayed(process_single_patient)
                      (patient_dir, INPUT_DICOM_DIR, OUTPUT_NPY_DIR) for patient_dir in os.listdir(INPUT_DICOM_DIR))

In [ ]:
#for patient_dir in os.listdir(INPUT_DICOM_DIR):
#    process_single_patient(patient_dir, INPUT_DICOM_DIR, OUTPUT_NPY_DIR) 